# PRETPROCESIRANJE 

In [4]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import os
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [556]:
DATA_PATH = os.path.join('Master_Data - Copy') 
actions = np.array(['Ja','Mogu','Riba','Sve','Momak','Devojka','Opasno','Kada','Zasto','Hvala'])
no_sequences = 20
sequence_length = 35
label_map = {label:num for num, label in enumerate(actions)}

# Trening, test, validacija skup

In [557]:
# Definisanje osoba po polu
genders = ['man', 'woman']
persons = {
    'man': ['person2', 'person4'],
    'woman': ['person1', 'person3', 'person5']
}

# Podela osoba na trening, test i validaciju
train_persons = {'man': ['person2'], 'woman': ['person5', 'person1']}
test_persons = {'woman': ['person3'], 'man':[]}
validation_persons = {'man': ['person4'], 'woman':[]}


sequences_train, labels_train = [], []
sequences_test, labels_test = [], []
sequences_validation, labels_validation = [], []

for gender in genders:
    for person in persons[gender]:
        for action in actions:
            action_path = os.path.join(DATA_PATH, gender, person, action)
            if not os.path.exists(action_path):
                continue
            for sequence in np.array(os.listdir(action_path)).astype(int):
                window = []
                for frame_num in range(sequence_length):
                    res = np.load(os.path.join(action_path, str(sequence), "{}.npy".format(frame_num)))
                    window.append(res)
                if person in train_persons[gender]:
                    sequences_train.append(window)
                    labels_train.append(label_map[action])
                elif person in validation_persons[gender]:
                    sequences_validation.append(window)
                    labels_validation.append(label_map[action])
                elif person in test_persons[gender]:
                    sequences_test.append(window)
                    labels_test.append(label_map[action])

In [558]:
X_train = np.array(sequences_train)
y_train = to_categorical(labels_train).astype(int)
X_test = np.array(sequences_test)
y_test = to_categorical(labels_test).astype(int)
X_val = np.array(sequences_validation)
y_val = to_categorical(labels_validation).astype(int)

In [3]:
#Cuvanje podataka da ne moramo svaki put da ih ucistavamo i razvrsavamo

In [2]:
def save_dataset(path, X_train, y_train, X_val, y_val, X_test, y_test):
    np.savez(path,
             X_train=X_train, y_train=y_train,
             X_val=X_val, y_val=y_val,
             X_test=X_test, y_test=y_test)
    print(f"Dataset saved to {path}")

In [956]:
save_dataset('my_dataset.npz', X_train, y_train, X_val, y_val, X_test, y_test)

Dataset saved to my_dataset.npz


In [559]:
print('Training samples:', X_train.shape[0])
print('Test samples:', X_test.shape[0])
print('Training class distribution:', np.bincount(np.argmax(y_train, axis=1)))
print('Test class distribution:', np.bincount(np.argmax(y_test, axis=1)))
print('Validation samples:', X_val.shape[0])
print('Val class distribution:', np.bincount(np.argmax(y_val, axis=1)))

Training samples: 600
Test samples: 200
Training class distribution: [60 60 60 60 60 60 60 60 60 60]
Test class distribution: [20 20 20 20 20 20 20 20 20 20]
Validation samples: 200
Val class distribution: [20 20 20 20 20 20 20 20 20 20]
